# Looks for Goldsmith peaks in the other analyses

In [1]:
files = {}
files['ink'] = 'spreadsheets/ink_v_ink/ink_v_ink_log_1e-05_tus.csv'
files['paper'] = 'spreadsheets/paper_v_paper/paper_v_paper_log_1e-05_tus.csv'
files['both'] = 'spreadsheets/both_v_both/both_v_both_log_1e-05_tus.csv'

In [2]:
import csv
masses = {}
for ids,filename in files.items():
    masses[ids] = []
    with open(filename,'r') as f:
        reader = csv.reader(f)
        heads = reader.next()
        for line in reader:
            q = float(line[-1])
            if q<=0.05:
                masses[ids].append(line[0])

In [8]:
goldsmith_file = 'spreadsheets/goldsmith/goldsmith_log_1e-05_tus.csv'
goldsmith_data = {}
with open(goldsmith_file,'r') as f:
    reader = csv.reader(f)
    gheads = reader.next()
    for line in reader:
        goldsmith_data[line[0]] = line

re-write the other files with goldsmith data appended where there is a match (just q<0.05 peaks)

In [20]:
res = {}
for fname in files:
    res[fname] = []
    with open(files[fname],'r') as f:
        reader = csv.reader(f)
        heads = reader.next()
        heads += gheads
        res[fname].append(heads)
        for line in reader:
            q = float(line[-1])
            if q<= 0.05:
                # try and find a goldsmith mz
                mz = float(line[0])
                best = None
                ee = 100
                for gmz in goldsmith_data.keys():
                    gmzf = float(gmz)
                    ppme = 1e6*abs(mz-gmzf)/mz
                    if ppme < ee:
                        best = gmz
                        ee = ppme
                if ee < 5:
                    line += goldsmith_data[best]
                    res[fname].append(line)

In [24]:
for fname in res:
    newfname = 'spreadsheets/goldsmith/{}_matches.csv'.format(fname)
    with open(newfname,'w') as f:
        writer = csv.writer(f)
        for line in res[fname]:
            writer.writerow(line)